<a href="https://colab.research.google.com/github/22046361/Coding--1-Homework-Submission/blob/main/saving_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
#Dont use scientific notation for numbers (e.g 1.003767687e-12)
pd.set_option('display.float_format','{:.5f}'.format)
np.set_printoptions(suppress=True)

## Collaborative Filtering



### Importing Dataset

In [ ]:
import pandas as pd

data = pd.read_csv('data/Preprocessed_data.csv')
pd.options.display.max_columns = 150

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# Check the size of the column
column_size = len(data['user_id'])
print("Size of user_id column:", column_size)

# Check the unique values in the column
unique_values = data['user_id'].unique()
print("Unique user IDs:", unique_values)

In [ ]:
data = data[(data['rating'] != 0) & (~data['rating'].isnull())]

I know by testing with the data, the dataset is too big for my computer to compute, so I will aim to reduce the size of the  dataset.  I first wanted to remove the ratings of 0, as I am not sure this would means that the use rated with a 0 or not rating the book. Therefore I will only keep a scale of rating 1 - >10.  I will then drop the rows of duplicates and the rows that have no book title.

In [ ]:
data.shape

In [ ]:
# Check for duplicate ratings by the same user for a book. Code from https://towardsdatascience.com/finding-and-removing-duplicate-rows-in-pandas-dataframe-c6117668631f#:~:text=To%20find%20duplicates%20on%20a,()%20method%20on%20the%20column.&text=The%20result%20is%20a%20boolean,identical%20to%20a%20previous%20one.
duplicate_ratings = data[data.duplicated(subset=['user_id', 'book_title'], keep=False)]

if len(duplicate_ratings) > 0:
    print("Duplicate ratings found by the same user for the following books:")
    print(duplicate_ratings[['user_id', 'book_title']])
else:
    print("No duplicate ratings found.")

In [ ]:
new_data = data.drop_duplicates(subset=['user_id', 'book_title'], keep='first')

In [ ]:
new_data.shape

In [ ]:
missing_values = new_data['book_title'].isnull().sum()
zeros_count = (new_data['book_title'] == 0).sum()

if missing_values > 0:
    print("There are", missing_values, "missing values in the 'book_title' column.")
else:
    print("There are no missing values in the 'book_title' column.")

if zeros_count > 0:
    print("There are", zeros_count, "zeros in the 'book_title' column.")
else:
    print("There are no zeros in the 'book_title' column.")

To use the model, the book_titles together with the ratings will be the key parameters. As the book titles are non numerical I would need to chang this to help with the compute.

In [ ]:
new_data = new_data.copy()
new_data.loc[:, 'book_id'] = pd.factorize(new_data['book_title'])[0]




# Assuming your DataFrame is named 'df' and the column with user IDs is 'user_id'
new_data['user_id'] = pd.factorize(new_data['user_id'])[0]

# Verify the updated values and size of the 'user_id' column
print("Size of user_id column:", len(new_data['user_id']))
print("Unique user IDs:", new_data['user_id'].unique())

In [ ]:
new_data.head

In [ ]:
df=new_data.drop(['Unnamed: 0','location','isbn','img_s','img_m','img_l','Summary','city','state'],axis=1)

In [ ]:
df.head

In [ ]:
df = df.drop(df[df['book_id'] == 0].index)

In [ ]:
df.head

In [ ]:
# Check the size of the column
column_size = len(df['user_id'])
print("Size of user_id column:", column_size)

# Check the unique values in the column
unique_values = df['user_id'].unique()
print("Unique user IDs:", unique_values)

In [ ]:
import pandas as pd
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
user_ids_subset = df['user_id'].unique()[:5000]
df_subset = df[df['user_id'].isin(user_ids_subset)]

In [ ]:
import numpy as np

user_ids = df_subset["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids = df_subset["book_id"].unique().tolist()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}

# Check the size of user_ids list
#num_unique_users = df_subset["user_id"].nunique()
#if len(user_ids) != num_unique_users:
#    raise ValueError("The size of user_ids list does not match the number of unique user IDs in the dataset.")

#df_subset.loc[:, "user"] = df_subset["user_id"].map(user2user_encoded)
#df_subset.loc[:, "book"] = df_subset["book_id"].map(book2book_encoded)

df_subset["user"] = df_subset["user_id"].map(user2user_encoded)
df_subset["book"] = df_subset["book_id"].map(book2book_encoded)



num_users = len(user2user_encoded)
num_books = len(book_encoded2book)
#df_subset.loc[:, "rating"] = df_subset["rating"].values.astype(np.float32)
df_subset["rating"] = df_subset["rating"].values.astype(np.float32)
min_rating = min(df_subset["rating"])
max_rating = max(df_subset["rating"])

print(
    "Number of users: {}, Number of books: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_books, min_rating, max_rating
    )
)

In [ ]:
# new code part 2

## Prepare training and validation data

df_subset = df_subset.sample(frac=1, random_state=42)
x = df_subset[["user", "book"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df_subset["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df_subset.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [ ]:
# Print the shapes of training data
print("Training data shapes:")
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

# Print the shapes of validation data
print("\nValidation data shapes:")
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)

In [ ]:
EMBEDDING_SIZE = 50


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_books, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_books = num_books
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.book_embedding = layers.Embedding(
            num_books,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.book_bias = layers.Embedding(num_books, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        book_vector = self.book_embedding(inputs[:, 1])
        book_bias = self.book_bias(inputs[:, 1])
        dot_user_book = tf.tensordot(user_vector, book_vector, 2)
        # Add all the components (including bias)
        x = dot_user_book + user_bias + book_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_books, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [ ]:
import numpy as np

user_ids = df_subset["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids = df_subset["book_id"].unique().tolist()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}

# Check the size of user_ids list
num_unique_users = df_subset["user_id"].nunique()
if len(user_ids) != num_unique_users:
    raise ValueError("The size of user_ids list does not match the number of unique user IDs in the dataset.")

df_subset.loc[:, "user"] = df_subset["user_id"].map(user2user_encoded)
df_subset.loc[:, "book"] = df_subset["book_id"].map(book2book_encoded)

num_users = len(user2user_encoded)
num_books = len(book_encoded2book)
df_subset.loc[:, "rating"] = df_subset["rating"].values.astype(np.float32)
min_rating = min(df_subset["rating"])
max_rating = max(df_subset["rating"])

print(
    "Number of users: {}, Number of books: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_books, min_rating, max_rating
    )
)



In [ ]:
#part 3


## Train the model based on the data split

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)


## Plot training and validation loss

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

probably not correct

In [ ]:
def recommend_movie(test_user_id, df, df_subset, book2book_encoded, user2user_encoded):
    # Let us get a user and see the top recommendations.
    books_read_by_user = df_subset[df_subset.user_id == test_user_id]
    books_not_read = df[
        ~df["book_id"].isin(books_read_by_user.book_id.values)
    ]["book_id"]
    books_not_read = list(
        set(books_not_read).intersection(set(book2book_encoded.keys()))
    )
    books_not_read = [[book2book_encoded.get(x)] for x in books_not_read]
    user_encoder = user2user_encoded.get(test_user_id)
    user_book_array = np.hstack(
        ([[user_encoder]] * len(books_not_read), books_not_read)
    )
    ratings = model.predict(user_book_array).flatten()
    top_ratings_indices = ratings.argsort()[-10:][::-1]
    recommended_book_ids = [
        book_encoded2book.get(books_not_read[x][0]) for x in top_ratings_indices
    ]

    print("Showing recommendations for user: {}".format(test_user_id))
    print("====" * 9)
    print("Books with high ratings from user")
    print("----" * 8)
    top_books_user = (
        books_read_by_user.sort_values(by="rating", ascending=False)
        .head(5)
        .book_id.values
    )
    df_rows = df[df["book_id"].isin(top_books_user)]
    for row in df_rows.itertuples():
        print(row.title, ":", row.genres)

    print("----" * 8)
    print("Top 10 book recommendations")
    print("----" * 8)
    recommended_books = df[df["book_id"].isin(recommended_book_ids)]
    for row in recommended_books.itertuples():
        print(row.title, ":", row.genres)

    print("\n\n\n")